In [5]:
import selenium
from selenium import webdriver
import pandas as pd 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time 
import re
import os
pd.options.mode.chained_assignment = None  # default='warn'


In [229]:
# Create driver
driver = webdriver.Chrome(ChromeDriverManager().install())

# Get the page url 
page_url = "https://transcripts.foreverdreaming.org/viewtopic.php?f=22&t=4411"
driver.get(page_url) # Open the page 
wait = WebDriverWait(driver, 10) # Wait until the page renders

[WDM] - Downloading: 100%|█████████████████| 8.41M/8.41M [00:07<00:00, 1.25MB/s]
/var/folders/_b/l_819g5924d_y00npn9gt60h0000gn/T/ipykernel_29310/2381540048.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [6]:
item_links = [item.get_attribute("href") for item in driver.find_elements(By.CLASS_NAME, 'postlink')] # Get links to the scripts 

NameError: name 'driver' is not defined

In [7]:
# Scrape scripts from links 
for item_link in item_links[]:
    driver.get(item_link) # Click the link  
    time.sleep(3) 
    l = driver.find_elements(By.TAG_NAME, 'p')
    for i in l: 
        print (i.text) 


SyntaxError: invalid syntax (1299288386.py, line 2)

In [53]:
# Load scripts
scripts = [s for s in sorted(os.listdir('data')) if s.startswith('season')] # List of scripts sorted 

In [ ]:
# characters['name'] = characters['full name'].apply(lambda x: x if 
#                                                    x.startswith(("Ms.", "Miss", "Mrs.", "Mr")) else x.split()[0]) 

In [96]:
# Clean dataframes 
def prepare_dataframe(df):
    df.rename(columns = {list(df)[0]:'Line'}, inplace = True) 

    df['Character'] = df['Line'].str.split(' ').str[0] # Create a column with first word from line 
    df['Line'] = df['Line'].str.split(n=1).str[1] # Delete the first word from line 
    df = df[df['Character'].str.endswith(':', na=False)] # Delete row if doesn't end with ":"
    
    # Change the order of columns 
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]

    # Capitalize character names 
    df['Character'] = df['Character'].str.capitalize()
    df['Character'] = df['Character'].str.replace(':', '')
    
    return df 

In [109]:
# Create list of dataframes 
seasons_list = []
 
# Append seasons scripts into the list
for i in range(len(scripts)):
    df = pd.read_csv("data/"+scripts[i])
    seasons_list.append(df)

# Prepare all the dataframes in the list 
seasons_list = [df.pipe(prepare_dataframe) for df in seasons_list]
seasons_list

[          Character                                               Line
 4           Lorelai              Please, Luke. Please, please, please.
 5              Luke           How many cups have you had this morning?
 6           Lorelai                                              None.
 7              Luke                                            Plus...
 8           Lorelai                         Five, but yours is better.
 ...             ...                                                ...
 17909       Lorelai             I just really need to tell Rory first.
 17910          Luke                  But you're happy about this news.
 17911       Lorelai  I might be. Maybe. It's just kind of a, wow I ...
 17913          Luke                                                Bye
 17916  Lorelai/rory                 You first! (They laugh) You first!
 
 [15271 rows x 2 columns],
          Character                                               Line
 4             Rory                 

In [111]:
# seasons_numbers = ['1', '2', '3', '4', '5', '6', '7']

# for i in seasons_numbers:
#     df['Season'] = str(i) 
#     seasons_list.append(df)
    
# seasons_list

In [110]:
# Merge dataframes
all_seasons = pd.concat(seasons_list, ignore_index=True)
all_seasons

,Character,Line
0,Lorelai,"Please, Luke. Please, please, please."
1,Luke,How many cups have you had this morning?
2,Lorelai,None.
3,Luke,Plus...
4,Lorelai,"Five, but yours is better."
...,...,...
115200,Rory,Well It wasn't very flattering.
115201,Lorelai,Well how will people know you're the world's g...
115202,Rory,I don't know.
115203,Lorelai,I guess they'll just have to read your stuff.


In [13]:
# Replace mispelled names 
all_seasons['Character'] = all_seasons['Character'].replace(['Loreai', 'Lorleai', 'Loreali', 'Miche', 'Sooki', 'Sookei', 'Christohper', 'Orelai',
                                                            'Richrad', 'Grandpa', 'Lorelail', 'Ory', 'Christoper', 'Lorlelai'], 
                                                            ['Lorelai', 'Lorelai', 'Lorelai', 'Michel', 'Sookie', 'Sookie', 'Christopher', 'Lorelai',
                                                            'Richard', 'Richard', 'Lorelai', 'Rory', 'Christopher', 'Lorelai'])

In [16]:
# Save output to csv 
all_seasons.to_csv(r'/Users/julkakubisa/gilmore_girls_network/data/Gilmore_Girls_Lines.csv', index=True)